# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import sqlite3

In [2]:
# Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite')
cur = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [3]:
cur.execute('''DROP TABLE contactInformation''')

In [4]:
# Your code here
cur.execute("""CREATE TABLE contactInformation (
                                           firstName TEXT,
                                           lastName TEXT,
                                           role TEXT,
                                           telephone FLOAT,
                                           street TEXT,
                                           city TEXT,
                                           STATE TEXT,
                                           zipcode TEXT)""")

In [7]:
cur.execute("""PRAGMA table_info(contactInformation)""").fetchall()

[(0, 'firstName', 'TEXT', 0, None, 0),
 (1, 'lastName', 'TEXT', 0, None, 0),
 (2, 'role', 'TEXT', 0, None, 0),
 (3, 'telephone', 'FLOAT', 0, None, 0),
 (4, 'street', 'TEXT', 0, None, 0),
 (5, 'city', 'TEXT', 0, None, 0),
 (6, 'STATE', 'TEXT', 0, None, 0),
 (7, 'zipcode', 'TEXT', 0, None, 0)]

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [12]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)
    
contacts = [{'firstName': 1,
  'lastName': 2,
  'role': 3,
  'telephone ': '2035687697',
  'street': '1672 Whitman Court',
  'city': 'Stamford',
  'state': 'CT',
  'zipcode ': '06995'}]


In [13]:
print([x[0] + ', ' for x in cur.execute('''SELECT * FROM contactInformation''').description])

['firstName, ', 'lastName, ', 'role, ', 'telephone, ', 'street, ', 'city, ', 'STATE, ', 'zipcode, ']


In [14]:
# Iterate over the contact list and populate the contactInfo table here
for c in contacts:
    value_string = [str(x) if isinstance(x, int) else '"'+x+'"' for x in c.values()]
    value_string = ', '.join(value_string)
    
    cur.execute(f'''INSERT INTO contactInformation 
                        ({', '.join(contacts[0].keys())})
                VALUES
                        ({value_string}) ''')

**Query the Table to Ensure it is populated**

In [15]:
# Your code here 
cur.execute('''SELECT * from contactInformation''').fetchall()

[('1',
  '2',
  '3',
  2035687697.0,
  '1672 Whitman Court',
  'Stamford',
  'CT',
  '06995')]

## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [70]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [83]:
cur.execute('''DROP TABLE grades''')

In [84]:
# Create the grades table
cur.execute('''
    CREATE TABLE grades
        (userId INT NOT NULL,
         courseId INT NOT NULL,
         grade TEXT,
         PRIMARY KEY (userId, courseId)
        )''')

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [141]:
# Find the duplicate entry
to_del = cur.execute(f'''SELECT * FROM contactInformation
                GROUP BY {', '.join(contacts[0].keys())} 
                HAVING COUNT(*) > 1''').fetchone()



In [142]:
# Delete the duplicate entry
keys = [x[0] for x in cur.description]
to_del = [str(x) if isinstance(x, int) else '"'+x+'"' for x in to_del]
kv = zip(keys, to_del)
kv = [' = '.join(x) for x in list(kv)]
kv = ' AND '.join(kv)
print(kv)
cur.execute(f'''DELETE FROM contactInformation WHERE {kv}''')

firstName = "Jane" AND lastName = "Evans" AND role = "student" AND telephone = 3259909290 AND street = "1461 Briarhill Lane" AND city = "Abilene" AND STATE = "TX" AND zipcode = "79602"


In [143]:
# Check that the duplicate entry was removed
cur.execute(f'''SELECT * FROM contactInformation
                GROUP BY {', '.join(contacts[0].keys())} 
                HAVING COUNT(*) > 1''').fetchone()

## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [146]:
# Update Ed's address
cur.execute('''UPDATE contactInformation
                SET street = "2910 Simpson Avenue",
                    city = "York",
                    state = "PA",
                    zipcode = "17403"
                WHERE firstName = "Ed" AND lastName = "Lyman"''')

In [149]:
# Query the database to ensure the change was made
cur.execute('''SELECT * FROM contactInformation WHERE firstName = "Ed" AND lastName = "Lyman"''').fetchall()

[('Ed',
  'Lyman',
  'student',
  5179695576,
  '2910 Simpson Avenue',
  'York',
  'PA',
  '17403')]

## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [150]:
# Your code here
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 